In [1]:
import pickle
import os
import numpy as np
import pandas as pd
import random

In [2]:
def load_pkl(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

# Process data

In [2]:
from components.DataProcessor import DataProcessor


file_path = 'COMP90086_2023_TLLdataset/train.csv'
test_file_path = 'COMP90086_2023_TLLdataset/test_candidates.csv'
save_path = 'dictionaries'
dataset_base_path = 'COMP90086_2023_TLLdataset'

processor = DataProcessor(file_path, test_file_path, save_path, dataset_base_path)
processor.load_data()
processor.split_data()
processor.generate_dictionaries()
processor.save_dictionaries()
processor.process_test_candidates()


Dictionaries saved to 'dictionaries/train_dict.pkl' and 'dictionaries/val_dict.pkl'
Dictionaries saved to dictionaries/test_dict.pkl


In [4]:
train_data = load_pkl('dictionaries/train_dict.pkl')
val_data = load_pkl('dictionaries/val_dict.pkl')
test_data = load_pkl('dictionaries/test_dict.pkl')

print('train data size: ', len(train_data))
print('val data size: ', len(val_data))
print('test data size: ', len(test_data))

train data size:  1600
val data size:  400
test data size:  2000


In [38]:
print('train data sample: ', val_data)

train data sample:  {'COMP90086_2023_TLLdataset/train/left/yfj.jpg': 'COMP90086_2023_TLLdataset/train/right/imq.jpg', 'COMP90086_2023_TLLdataset/train/left/eji.jpg': 'COMP90086_2023_TLLdataset/train/right/jbv.jpg', 'COMP90086_2023_TLLdataset/train/left/rfu.jpg': 'COMP90086_2023_TLLdataset/train/right/jex.jpg', 'COMP90086_2023_TLLdataset/train/left/lta.jpg': 'COMP90086_2023_TLLdataset/train/right/gph.jpg', 'COMP90086_2023_TLLdataset/train/left/qpb.jpg': 'COMP90086_2023_TLLdataset/train/right/ynk.jpg', 'COMP90086_2023_TLLdataset/train/left/qjm.jpg': 'COMP90086_2023_TLLdataset/train/right/qkz.jpg', 'COMP90086_2023_TLLdataset/train/left/mgi.jpg': 'COMP90086_2023_TLLdataset/train/right/ggb.jpg', 'COMP90086_2023_TLLdataset/train/left/wng.jpg': 'COMP90086_2023_TLLdataset/train/right/xsr.jpg', 'COMP90086_2023_TLLdataset/train/left/awg.jpg': 'COMP90086_2023_TLLdataset/train/right/nls.jpg', 'COMP90086_2023_TLLdataset/train/left/rda.jpg': 'COMP90086_2023_TLLdataset/train/right/vht.jpg', 'COMP9008

# Extract features

In [3]:
from components.FeatureExtractor import FeatureExtractor

# define the output dir & models you want to train here
base_output_dir = "feat"
models_to_train = ["resnet50", "resnet101", "resnet152", "densenet201", "vgg16", "mobilenet"]

train_data = load_pkl('dictionaries/train_dict.pkl')
val_data = load_pkl('dictionaries/val_dict.pkl')
test_data = load_pkl('dictionaries/test_dict.pkl')

# for model_name in models_to_train:
#     extractor = FeatureExtractor(model_name, base_output_dir)
#     for set_name, data_dict in [("train_data", train_data), ("val_data", val_data)]:
#         extractor.extract_and_save_features(data_dict, set_name)

# model_name = "resnet50"
# extractor = FeatureExtractor(model_name, base_output_dir)
# extractor.extract_and_save_features_for_test(test_data, "test_data")

2023-10-17 19:25:31.826284: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 61ms/step


# Create anchor, positive and negative examples

In [3]:
def load_saved_features(model_name, set_name, side, base_dir="feat"):
    file_path = os.path.join(base_dir, model_name, set_name, f"{set_name}_{side}_features.pkl")
    with open(file_path, 'rb') as f:
        features = pickle.load(f)
    return features

def generate_negative_samples(loaded_features, num_negative_samples):
    all_features = np.concatenate(list(loaded_features.values()))
    negative_samples = []
    for _ in range(num_negative_samples):
        random_idx = np.random.choice(len(all_features))
        negative_samples.append(all_features[random_idx])
    return np.array(negative_samples)

base_dir = "feat"
model_name = "resnet50"  
set_name = "train_data" 

# Load anchor and positive samples
anchor_features = load_saved_features(model_name, set_name, "left", base_dir)
positive_features = load_saved_features(model_name, set_name, "right", base_dir)

# Generate negative samples
num_negative_samples = len(anchor_features)
negative_features = generate_negative_samples(anchor_features, num_negative_samples)

# Prepare training data
anchors = np.concatenate(list(anchor_features.values()))
positives = np.concatenate(list(positive_features.values()))
negatives = negative_features

# Load model and Train

In [4]:
from components.SimilarityModel import SimilarityModel

input_shape = (len(anchor_features['COMP90086_2023_TLLdataset/train/left/mry.jpg'].flatten()),)
image_similarity_model = SimilarityModel(input_shape)
image_similarity_model.train(anchors, positives, negatives)
# most_similar_indices = image_similarity_model.find_most_similar(val_left, candidates)

Epoch 1/50


2023-10-17 19:50:21.319232: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


50/50 [==============================] - 0s 2ms/step - loss: 1.1015
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3813
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2182
Epoch 4/50
50/50 [==============================] - 0s 3ms/step - loss: 0.1561
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1344
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1223
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1420
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1177
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1230
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1064
Epoch 11/50
50/50 [==============================] - 0s 2ms/step - loss: 0.0917
Epoch 12/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1111
Epoch 13/50
50/50 [==============================] - 0s 2ms/

In [18]:
from components.SimilarityEvaluator import SimilarityEvaluator

model_name = "resnet50"
val_set_name = "val_data"
evaluator = SimilarityEvaluator(model_name, val_set_name)

val_dict = evaluator.load_pkl('dictionaries/val_dict.pkl')

accuracy = evaluator.evaluate_accuracy(image_similarity_model, val_dict)
print('acc: ', accuracy)

acc:  0.4725


# Get predictions

In [6]:
test_left = load_saved_features("resnet50", "test_data", "left", base_dir)
test_right = load_saved_features("resnet50", "test_data", "right", base_dir)

In [17]:
import csv

# Create a dictionary to store similarity scores
similarity_scores = {}

# Iterate through the rows of the CSV file
with open('COMP90086_2023_TLLdataset/test_candidates.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        left_image = row['left']
        right_images = [row[f'c{i}'] for i in range(20)]  # Include only 20 right images
        
        scores = []  # List to store similarity scores for the current left image
        
        # Iterate through the right images
        for right_image in right_images:
            left_image_path = os.path.join('COMP90086_2023_TLLdataset/test/left', left_image)
            right_image_path = os.path.join('COMP90086_2023_TLLdataset/test/right', right_image)
            
            left_features = test_left[left_image_path + '.jpg']
            right_features = test_right[right_image_path + '.jpg']
            
            # Compute the similarity between the left and right images
            similarity = image_similarity_model.compute_similarity(left_features, right_features)
            scores.append(similarity)
        
        # Add the scores to the dictionary with the left image as the key
        similarity_scores[left_image] = scores

# Write the similarity scores to a new CSV file
with open('similarity_scores.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header row
    header_row = ['left'] + [f'c{i}' for i in range(20)]  # Include only 20 columns
    writer.writerow(header_row)

    # Write the similarity scores for each left image, limiting to 20 columns
    for left_img, scores in similarity_scores.items():
        writer.writerow([left_img] + scores)
